In [67]:
import requests
import folium
import geojson
import json
import folium.plugins as plugins
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import time
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [215]:
m = folium.Map(location=[59.387633, -12.548914])
m.fit_bounds([[59.387633, -12.548914],[48.844622, 2.369097]])

In [216]:
all_stations = 'https://admiraltyapi.azure-api.net/uktidalapi/api/V1/Stations'
headers = {'Ocp-Apim-Subscription-Key':'68e43f02f96342dbb2e4915455aa995c'}
response = requests.get(all_stations, headers=headers)

In [217]:
tidal_geojson = json.loads(response.text)
heat=[]
for feature in tidal_geojson["features"]:
    name = feature["properties"]["Name"]
    continuous_heights = feature["properties"]["ContinuousHeightsAvailable"]
    popup_text = 'Name: ' + name
    coords = feature["geometry"]["coordinates"]
    coords = [coords[1],coords[0]]
    heat = heat + [coords]
plugins.HeatMap(heat).add_to(m)
m


In [117]:
all_station_ids = json.loads((requests.get(all_stations, headers=headers)).text)
all_station_ids = (pd.DataFrame.from_records(all_station_ids["features"]))
all_station_ids = all_station_ids['properties']
all_station_ids = (pd.DataFrame.from_records(all_station_ids))
all_station_ids["Id"]

0       0324
1       0001
2       0308
3       0321
4      0321A
5       0323
6       0246
7       0245
8       0244
9       0207
10      0208
11      0209
12      0221
13      0322
14      0659
15      0657
16      0628
17     0628A
18      0629
19      0643
20      0644
21      0651
22      0652
23      0653
24      0658
25      0115
26     0115A
27      0116
28      0332
29      0330
       ...  
577     0410
578     0411
579     0412
580     0413
581    0406A
582     0406
583     0407
584     0355
585    0353A
586     0353
587     0352
588     0351
589    0349A
590     0349
591     0341
592     0339
593     0368
594    0368A
595     0277
596     0275
597    0273B
598    0273A
599     0273
600     0272
601     0271
602     0274
603     0276
604     0279
605     0281
606     0282
Name: Id, Length: 607, dtype: object

In [124]:
#response_jsons = []
station_heights = []
print(len(all_station_ids))
for station in all_station_ids["Id"]:
    get_station_tides = 'https://admiraltyapi.azure-api.net/uktidalapi/api/V1/Stations/'+station+'/TidalEvents'
    headers = {'Ocp-Apim-Subscription-Key':'68e43f02f96342dbb2e4915455aa995c'}
    response = requests.get(get_station_tides, headers=headers)
    response_json = json.loads(response.text)
    all_heights = []
    for item in response_json:
        if 'Height' in item.keys():
            all_heights.append(item['Height'])
    station_heights.append({'id': station, 'heights': all_heights})
    time.sleep(.200)
print(len(station_heights))    

607
607


In [218]:
max_heights = []
min_heights = []
for station in station_heights:
    heights = station['heights']
    if (len(heights) != 0) :
        station_id = station['id']
        max_height = max(heights)
        min_height = min(heights)
        max_heights.append({'id': station_id, 'height': max_height})
        min_heights.append({'id': station_id, 'height': min_height})

In [219]:
max_heights = pd.DataFrame.from_records(max_heights)
max_heights = max_heights.sort_values('height', ascending=False)
max_height_station = max_heights.head(1).iloc[0]['id']
max_height_value = max_heights.head(1).iloc[0]['height']
get_station = 'https://admiraltyapi.azure-api.net/uktidalapi/api/V1/Stations/'+max_height_station
headers = {'Ocp-Apim-Subscription-Key':'68e43f02f96342dbb2e4915455aa995c'}
response = requests.get(get_station, headers=headers)
coords = json.loads(response.text)["geometry"]["coordinates"]
coords = [coords[1], coords[0]]
folium.Marker(coords,
              popup="Max Height this week: " + str(max_height_value)+" Belongs to: " + max_height_station).add_to(m)
m


In [220]:
min_heights = pd.DataFrame.from_records(min_heights)
min_heights = min_heights.sort_values('height', ascending=True)
min_height_station = min_heights.head(1).iloc[0]['id']
min_height_value = min_heights.head(1).iloc[0]['height']
get_station = 'https://admiraltyapi.azure-api.net/uktidalapi/api/V1/Stations/'+min_height_station
headers = {'Ocp-Apim-Subscription-Key':'68e43f02f96342dbb2e4915455aa995c'}
response = requests.get(get_station, headers=headers)
coords = json.loads(response.text)["geometry"]["coordinates"]
coords = [coords[1], coords[0]]
folium.Marker(coords,
              popup="Min Height this week: " + str(min_height_value)+" Belongs to: " + min_height_station).add_to(m)
m

In [221]:
m.save("test_map.html")